In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(user='gpadmin',
                        password='gpadmin',
                        host='srv8-hausma.net.billing.ru',
                        port="5432",
                        database='ndr_spa_db_paas')
df = pd.DataFrame()

with conn.cursor() as cur:

# Просмотрело пользователей
    cur.execute("""
 SELECT COALESCE (t_spa_merging.merged_spa_id,t_dashboard_data_spa.spa_id) AS "User",
       t_dashboard_data_spa."timestamp" AS "Date",
       t_dashboard_data_spa.custom_event_id AS "Action"
        ,t_channel.channelname as t_channel_channelname,
       t_ua_browser.family as t_ua_browser_family,
       t_ua_os.family as t_ua_os_family
FROM metrics.dashboard_data_spa as t_dashboard_data_spa
         INNER JOIN metrics.ua_os as t_ua_os ON t_dashboard_data_spa.os_id = t_ua_os.id
         INNER JOIN metrics.ua_browser as t_ua_browser ON t_dashboard_data_spa.browser_id = t_ua_browser.id
         INNER JOIN metrics.channel as t_channel ON t_dashboard_data_spa.channel_name_typeid = t_channel.channel_name_typeid
         LEFT JOIN metrics.spa_merging t_spa_merging ON t_dashboard_data_spa.spa_id = t_spa_merging.spa_id
WHERE
        t_dashboard_data_spa.date_timestamp >= 1650326400.0
  AND t_dashboard_data_spa.date_timestamp <= 1653004799.999
  AND t_dashboard_data_spa.counterid = 28
ORDER BY 1, 2, 3
""")


    data = cur.fetchall()
    columns = list(cur.description)
    datacolumns=[c[0] for c in columns]
    df = pd.DataFrame(data=data, columns = datacolumns)
    
#df.head(100)  
df

In [ ]:
# 1 ШАГ - ВЫГРУЗКА ДАННЫХ

#выгружаем дата сет из csv
# import pandas as pd
# df = pd.read_csv('C:/Users/Maria.Ruchkina/Desktop/Исходные данные2.csv', encoding='windows-1251', sep=';')

# df['Date'] = pd.to_datetime(df['Date'])
# df['Action'] = df['Action'].astype(str)

#сортируем данные сначала по пользователю, зачем, для каждого пользователя - по дате
df=df.sort_values(by=['User', 'Date', 'Action'])

df.reset_index(inplace = True, drop = True)

#добавляем столбцы - Случай
df["Case"] = ''
#добавляем столбцы - Шаг
df["Step"] = ''
#добавляем столбец - Первый раз
df["FirstTime"] = ''

df.to_csv("исходные данные.csv", sep=';', encoding='windows-1251')
df

In [ ]:
#создаем список шагов

steps =[-5430214541187537878, -5514793887192677486]


In [10]:
# 2 ШАГ - ДЛЯ СТРОГОГО ПОРЯДКА - РАЗМЕЧАЕМ СОБЫТИЯ В ДАТА СЕТЕ

# caseNumber - счетчик для случаев
caseNumber = 1

# счетчик для шагов
stepNumber = 0

# цикл по шагам воронки
for step in steps:
    print (stepNumber)    
    # цикл по списку событий
    for i, row in df.iterrows():
        
        # работаем с записью только если для нее не определен шаг
        if row['Step'] == '':
            
            # если у записи событие = текущему шагу воронки
            if row['Action'] == step:
                
                # Пропускаем текущее событие, если:
                # Это не первая запись ИЛИ у предыдущей записи тот же юзер, событие и для него определен тот же шаг или шаг не определен
                # Это необходимо, так как если записи о событии от одного и того же пользователя идут подряд, мы считаем это одним и тем же шагом, а также для для того, чтобы не пропустить шаг, если для 2 шагов указано одно и то же событие
                
                #if i==0 or (df.iloc[i-1]['User'] == row['User'] and df.iloc[i-1]['Action'] == step and (df.iloc[i-1]['Step'] == (stepNumber+1) or df.iloc[i-1]['Step'] == '')) == False:
                    
                    # если текущий шаг - первый, фиксируем случай и шаг
                    if stepNumber == 0:
                        # если предыдущий шаг - с таким же Action и User и номером шага и следующий шаг - с другим Action
                        if df.iloc[i-1]['Action']==step and df.iloc[i-1]['User'] == row['User'] and df.iloc[i-1]['Step'] == stepNumber+1 and step != steps[stepNumber+1]:
                                df.at[i, 'Step'] = stepNumber+1
                                df.at[i, 'Case'] = df.iloc[i-1]['Case']
                                df.at[i, 'FirstTime']=False
                        elif i == 0 or df.iloc[i-1]['User'] != row['User'] or df.iloc[i-1]['Action'] != step:
                            df.at[i, 'Step'] = stepNumber+1
                            df.at[i, 'Case'] = caseNumber
                            df.at[i, 'FirstTime']=True
                            caseNumber = caseNumber + 1
                    # если текущий шаг - не первый
                    else:
                        if df.iloc[i-1]['Action']==steps[stepNumber-1] and df.iloc[i-1]['User'] == row['User'] and df.iloc[i-1]['Step']==stepNumber:
                            df.at[i, 'Case'] = df.iloc[i-1]['Case']
                            df.at[i, 'Step'] = stepNumber+1
                            df.at[i, 'FirstTime']=True
                        elif df.iloc[i-1]['Action']==step and df.iloc[i-1]['User'] == row['User'] and df.iloc[i-1]['Step']==stepNumber+1 and ((stepNumber+1) == len(steps) or step != steps[stepNumber+1]):
                            df.at[i, 'Step'] = stepNumber+1
                            df.at[i, 'Case'] = df.iloc[i-1]['Case']
                            df.at[i, 'FirstTime']=False
                       
    stepNumber = stepNumber + 1
                        
df_res=df[df.FirstTime == True]
df.to_csv("размеченные данные.csv", sep=';', encoding='windows-1251')
df

0
1


,User,Date,Action,t_channel_channelname,t_ua_browser_family,t_ua_os_family,Case,Step,FirstTime
0,80001,1652338224,-7120242342588305733,SPA WebSDK ver. 1.0,Chrome,Windows,,,
1,80001,1652338242,5643009978292827356,SPA WebSDK ver. 1.0,Chrome,Windows,,,
2,80001,1652338243,6214921114045282468,SPA WebSDK ver. 1.0,Chrome,Windows,,,
3,80001,1652338269,2952184785219546748,SPA WebSDK ver. 1.0,Chrome,Windows,,,
4,80001,1652338273,6214921114045282468,SPA WebSDK ver. 1.0,Chrome,Windows,,,
...,...,...,...,...,...,...,...,...,...
187,210011,1652870885,2828163426469846670,SPA WebSDK ver. 1.0,Chrome,Windows,,,
188,210011,1652870887,7056222391076071694,SPA WebSDK ver. 1.0,Chrome,Windows,,,
189,210011,1652870890,3020348873013704224,SPA WebSDK ver. 1.0,Chrome,Windows,,,
190,210011,1652870894,3020348873013704224,SPA WebSDK ver. 1.0,Chrome,Windows,,,


In [11]:
# ФОРМИРУЕМ СЛУЧАИ

cases_list = pd.DataFrame({'Case': df_res.Case.unique()})
cases_list["User"]=''

step_number = 0

for step in steps:
    step_number = step_number + 1
    if step_number != 1:
        cases_list["Diff_Step"+str(step_number)] = ''
    cases_list["Step"+str(step_number)] = ''
    
for i, case in cases_list.iterrows():
    step_number = 0
    for step in steps:
        step_number = step_number + 1
        step_row = df_res[(df_res['Case'] == case['Case']) & (df_res['Step'] == step_number)]
        if step_row.empty == False:
            cases_list.at[i, "Step" + str(step_number)]=step_row.iloc[0].Date
            cases_list.at[i, 'User']=step_row.iloc[0].User
            if step_number != 1:
                cases_list.at[i, "Diff_Step" + str(step_number)] = (cases_list.at[i, "Step" + str(step_number)] - cases_list.at[i, "Step" + str(step_number-1)])
                
cases_list

,Case,User,Step1,Diff_Step2,Step2
0,1,80001,1652338315,,
1,2,80001,1652338429,,
2,3,80001,1652338511,,
3,4,80006,1652338391,,
4,5,80006,1652338509,,
5,6,80011,1652338609,,
6,7,80011,1652338640,,
7,8,80011,1652338676,,
8,9,90001,1652433482,,
9,10,90001,1652433498,3,1652433501


In [12]:
# ОТОБРАЖАЕМ МЕТРИКИ

step_number = 0

for step in steps:
    step_number = step_number + 1    
    print("Шаг "+ str(step_number)+":")
    step_df=cases_list[cases_list["Step"+str(step_number)] != '']
    print("Количество случаев ="+ str(step_df.shape[0]))
    print("Количество пользователей = " + str(step_df.User.unique().shape[0]))
    if step_number != 1:
        print("Среднее время после предыдущего шага = " + str(step_df["Diff_Step" + str(step_number)].mean()))
        print("Среднее время после предыдущего шага (медиана) = " + str(step_df["Diff_Step" + str(step_number)].median()))

Шаг 1:
Количество случаев =12
Количество пользователей = 5
Шаг 2:
Количество случаев =1
Количество пользователей = 1
Среднее время после предыдущего шага = 3.0
Среднее время после предыдущего шага (медиана) = 3.0
